# Device-specific field calibration

## $N_0$ Parameter Calibration

The calibration of a Cosmic-Ray Neutron Probe (CRNP) is an essential step to ensure accurate soil moisture measurements. The CRNP operates by counting fast neutrons produced from cosmic rays, which are predominantly moderated by water molecules in the soil. The parameter $N_0$ is a device-specific constant that signifies the neutron count rate under zero soil moisture conditions. 

$\theta(N) =\frac{a_0}{(\frac{N}{N_0}) - a_1} - a_2 $ (Desilets et al., 2010).

## Determining field soil moisture

### Soil sampling layout

14 cores were collected from different distances. In this example each soil sample was split into four depth segments: 0-5 cm, 5-10 cm, 10-25 cm, and 25-40 cm. Soil samples were processed and soil moisture was determined using the thermo-gravimetric method.

<img src="../../../img/layout.png" style="max-width:500px">

Figure 1. Horizontal layout and vertical layout used in this particular example calibration, it can be customized by the user depending on their needs.

### Template for data collection

[Download the following template](https://github.com/soilwater/crnpy/blob/main/docs/examples/calibration/template.xlsx) spreadsheet for collecting soil samples data:

<img src="../../../img/template.png" style="max-width:500px">


In [1]:
# Importing required libraries
from scipy.optimize import root
import pandas as pd
import numpy as np
from crnpy import crnpy

# Import dataframes 

# Load the soil samples data and the CRNP dataset using pandas
df_soil = pd.read_csv("https://raw.githubusercontent.com/soilwater/crnpy/main/docs/examples/calibration/soil_data.csv")

# Load the station data
df_station = pd.read_csv("https://raw.githubusercontent.com/soilwater/crnpy/main/docs/examples/calibration/station_data.csv", skiprows=[0,2,3])


### Sample processing

For each sample it is required to know the bulk density ($\rho_\beta$) and the volumetric water content ($\theta_v$). See the details of the calculation used in the [filled example](https://github.com/soilwater/crnpy/blob/main/docs/examples/calibration/soil_data.csv). 

### Field average

Using the function [`nrad_weight()`](../../../reference/#crnpy.crnpy.nrad_weight) the weights corresponding to each soil sample will be computed considering air-humidity, sample depth, distance from station and bulk density.
Station data is used to calculate the absolute humidty from temperature an relative humidity data with [`estimate_abs_humidity()`](../../../reference/#crnpy.crnpy.estimate_abs_humidity).

In [2]:
#  Parse dates
df_station['TIMESTAMP'] = pd.to_datetime(df_station['TIMESTAMP'])

# Define dates
deployment_date = df_station['TIMESTAMP'].iloc[0]
# Soil sampling timestamp
calibration_start = pd.to_datetime("2021-10-22 08:00")
calibration_end = pd.to_datetime("2021-10-22 16:00")


# Filter data matching the sampling date
df_station_calib = df_station[(df_station['TIMESTAMP'] > calibration_start) & (df_station['TIMESTAMP'] < calibration_end)].copy()

#Calculate absolute humidity
df_station_calib['abs_h'] = crnpy.estimate_abs_humidity(df_station_calib['relative_humidity_Avg'], df_station_calib['air_temperature_Avg'])

# Compute the weights of each sample for the field average (https://en.wikipedia.org/wiki/Weighted_arithmetic_mean)
nrad_weights = crnpy.nrad_weight(df_station_calib['abs_h'].mean(), df_soil['theta_v'], df_soil['distance_from_station'], (df_soil['bottom_depth']+df_soil['top_depth'])/2, rhob=df_soil['bulk_density'].mean())

field_theta_v = np.sum(df_soil['theta_v']*nrad_weights)
field_bulk_density = np.sum(df_soil['bulk_density']*nrad_weights)
print(f"Field Volumetric Water content: {round(field_theta_v,3)}")

Field Volumetric Water content: 0.263


## Neutron count processing

Following a similar approach as the example for [Stationary CRNP](../../stationary/example_RDT_station/) neutron counts recorded while the field sampling was done will be corrected.

In [3]:
df = df_station_calib.copy()
# Set timestamp as index
df.set_index(df['TIMESTAMP'], inplace=True)
df.head()


,TIMESTAMP,RECORD,station,farm,field,latitude,longitude,altitude,battery_voltage_Min,PTemp_Avg,...,wind_speed_gust_Max,air_temperature_Avg,vapor_pressure_Avg,barometric_pressure_Avg,relative_humidity_Avg,humidity_sensor_temperature_Avg,tilt_north_south_Avg,tilt_west_east_Avg,NDVI_Avg,abs_h
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2021-10-22 09:00:00,2021-10-22 09:00:00,706,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.77,17.33,...,8.17,11.68,10.18,965,73.95,13.38,-0.775,0.975,0.26,7.719789
2021-10-22 10:00:00,2021-10-22 10:00:00,707,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.66,22.37,...,7.58,15.10,10.98,964,64.33,17.40,-0.775,1.250,0.266,8.293183
2021-10-22 11:00:00,2021-10-22 11:00:00,708,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.60,25.88,...,8.05,18.30,10.98,964,52.10,21.20,-0.850,1.150,0.265,8.140162
2021-10-22 12:00:00,2021-10-22 12:00:00,709,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.58,27.63,...,8.10,20.75,10.85,963,44.40,23.80,-0.800,1.000,0.262,8.010780
2021-10-22 13:00:00,2021-10-22 13:00:00,710,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.59,27.94,...,10.00,21.83,11.10,962,42.25,24.93,-1.575,1.650,0.256,8.114954


#### Computing total neutron counts



In [4]:
df['total_counts'] = crnpy.compute_total_raw_counts(df[['counts_1_Tot','counts_2_Tot']], nan_strategy='average')


#### Incomming neutron flux
Download the neutron flux for all the experiment, setting the reference value as the value when the station was deployed.

In [5]:
# Find stations with cutoff rigidity similar to the one estimated by lat,lon, 
# Filtering the time window from experiment setup to the end of the calibration
crnpy.find_neutron_monitor(crnpy.cutoff_rigidity(39.1, -96.6), start_date = deployment_date, end_date = calibration_end)

#Download data for one of the similar stations and add to df
incoming_neutrons = crnpy.get_incoming_neutron_flux(deployment_date, calibration_end, station="DRBS", utc_offset=-5)
df['incoming_flux']=crnpy.interpolate_incoming_flux(incoming_neutrons, timestamps=df['TIMESTAMP'])
ref_incoming_flux = incoming_neutrons.iloc[0]
df['corrected'] = crnpy.incoming_flux_correction(df['total_counts'], incoming_neutrons=df['incoming_flux'])



Select a station with an altitude similar to that of your location. For more information go to: 'https://www.nmdb.eu/nest/help.php#helpstations

Your cutoff rigidity is 2.87 GV
     STID     NAME     R  Altitude_m  Period available
13   DRBS  Dourbes  3.18         225              True
40   NEWK   Newark  2.40          50              True
28  KIEL2   KielRT  2.36          54              True


C:\Users\jperaza\AppData\Local\anaconda3\envs\crnpy\lib\site-packages\crnpy\crnpy.py:473: UserWarning: Reference incoming neutron flux not provided. Using first value of incoming neutron flux.
  warnings.warn('Reference incoming neutron flux not provided. Using first value of incoming neutron flux.')


#### Atmospheric correction

The atmospheric correction factors will correct neutron counts for atmospheric pressure and absolute humidity changes.

In [6]:
# Fill NaN values in atmospheric data
df[['pressure', 'RH', 'T']] = crnpy.fill_missing_atm(df[['barometric_pressure_Avg', 'relative_humidity_Avg', 'air_temperature_Avg']])

# Correct count by atmospheric variables and incoming flux
df['corrected'] = crnpy.humidity_correction(df['corrected'],humidity=df['RH'], temp=df['T'], Aref=0)
df['corrected'] = crnpy.pressure_correction(df['corrected'], pressure=df['pressure'], Pref=976, L=130)


In [7]:
print(f"Mean corrected neutron count during sampling: {int(df['corrected'].mean())}")


Mean corrected neutron count during sampling: 1537


## Solving the equation for $N_0$

Previous steps estimated the a field volumetric water content of `0.256` and an average neutron count of `1538`. Using [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html) $N_0$ is estimated given the observed value of $\theta_v$ and neutron counts.

In [8]:
# Define the function for which we want to find the roots
VWC_func = lambda N0 : crnpy.counts_to_vwc(df['corrected'].mean(), N0, bulk_density=field_bulk_density, Wlat=0.03, Wsoc=0.01) - field_theta_v

# Make an initial guess for N0
N0_initial_guess = 1000

# Find the root
sol = int(root(VWC_func, N0_initial_guess).x)

# Print the solution
print(f"The solved value for N0 is: {sol}")


The solved value for N0 is: 2636


## References: 
Desilets, D., Zreda, M., & Ferré, T. P. (2010). Nature's neutron probe: Land surface hydrology at an elusive scale with cosmic rays. Water Resources Research, 46(11).

Dong, J., & Ochsner, T. E. (2018). Soil texture often exerts a stronger influence than precipitation on mesoscale soil moisture patterns. Water Resources Research, 54(3), 2199-2211.

Patrignani, A., Ochsner, T. E., Montag, B., & Bellinger, S. (2021). A novel lithium foil cosmic-ray neutron detector for measuring field-scale soil moisture. Frontiers in Water, 3, 673185.